In [16]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, SpatialDropout1D
import numpy as np
import pickle
import os

# Load training data from text file
def load_training_data(file_path):
    texts = []
    labels = []
    label_map = {"Obtainable": 0, "Operable": 1, "Update": 2}  # Define label mapping
    with open(file_path, 'r') as file:
        for line in file:
            text, label = line.strip().split(',')
            texts.append(text)
            labels.append(label_map[label])  # Map label string to integer
    return texts, labels

# Sample dataset (for testing)
texts, labels = load_training_data('training_data.txt')

# Parameters
vocab_size = 5000
embedding_dim = 100
max_length = 20
trunc_type = 'post'
padding_type = 'post'
oov_tok = "<OOV>"

# Tokenization and padding
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(texts)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(texts)
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

# Preparing labels
labels = np.array(labels)

# Building the model
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100))
model.add(Dense(3, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Training the model
model.fit(padded_sequences, labels, epochs=20, verbose=2)

# Save the tokenizer for later use
tokenizer_path = "compiler/tokenizer.pickle"
os.makedirs(os.path.dirname(tokenizer_path), exist_ok=True)
with open(tokenizer_path, 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Save the model in the native Keras format
model.export('compiler')
model.save("compiler/SaveState.keras")


# Function to classify new requests
def classify_request(request):
    seq = tokenizer.texts_to_sequences([request])
    padded = pad_sequences(seq, maxlen=max_length, padding=padding_type, truncating=trunc_type)
    prediction = model.predict(padded)
    return np.argmax(prediction, axis=1)[0]

# Example usage
if __name__ == "__main__":
    new_request = "I am the best"
    request_type = classify_request(new_request)
    request_types = ["Obtainable", "Operable", "Update"]
    print(f"Request type: {request_types[request_type]}")


Epoch 1/20
4/4 - 4s - 892ms/step - accuracy: 0.3394 - loss: 1.1011
Epoch 2/20
4/4 - 0s - 24ms/step - accuracy: 0.3578 - loss: 1.0976
Epoch 3/20
4/4 - 0s - 22ms/step - accuracy: 0.3578 - loss: 1.0974
Epoch 4/20
4/4 - 0s - 23ms/step - accuracy: 0.3578 - loss: 1.0946
Epoch 5/20
4/4 - 0s - 26ms/step - accuracy: 0.3578 - loss: 1.0964
Epoch 6/20
4/4 - 0s - 35ms/step - accuracy: 0.3761 - loss: 1.0893
Epoch 7/20
4/4 - 0s - 22ms/step - accuracy: 0.4771 - loss: 1.0812
Epoch 8/20
4/4 - 0s - 23ms/step - accuracy: 0.5688 - loss: 1.0344
Epoch 9/20
4/4 - 0s - 22ms/step - accuracy: 0.5963 - loss: 0.9029
Epoch 10/20
4/4 - 0s - 41ms/step - accuracy: 0.6606 - loss: 0.7174
Epoch 11/20
4/4 - 0s - 22ms/step - accuracy: 0.6606 - loss: 0.6245
Epoch 12/20
4/4 - 0s - 23ms/step - accuracy: 0.7064 - loss: 0.5722
Epoch 13/20
4/4 - 0s - 22ms/step - accuracy: 0.6881 - loss: 0.5408
Epoch 14/20
4/4 - 0s - 25ms/step - accuracy: 0.8257 - loss: 0.4364
Epoch 15/20
4/4 - 0s - 82ms/step - accuracy: 0.8257 - loss: 0.3433
Epo

INFO:tensorflow:Assets written to: compiler\assets


Saved artifact at 'compiler'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 20), dtype=tf.float32, name='keras_tensor_105')
Output Type:
  TensorSpec(shape=(None, 3), dtype=tf.float32, name=None)
Captures:
  2291018700304: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2291018701264: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2291017972624: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2291018695120: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2291018698960: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2291018697040: TensorSpec(shape=(), dtype=tf.resource, name=None)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 224ms/step
Request type: Update


In [17]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle

# Load the compiled Keras model
model = load_model('compiler/SaveState.keras')

# Export the model to the SavedModel format
# model.save('tf_model')

# Convert the SavedModel to TFLite format with experimental flags
converter = tf.lite.TFLiteConverter.from_saved_model('compiler')
converter.experimental_enable_resource_variables = True
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]
converter._experimental_lower_tensor_list_ops = False

# Perform the conversion
tflite_model = converter.convert()

# Save the TFLite model
with open('compiler/intentionAi.tflite', 'wb') as f:
    f.write(tflite_model)

print("AI model built, converted and saved successfully!")


AI model built, converted and saved successfully!


In [18]:
from tensorflow.keras.models import model_from_json

# Load the compiled Keras model
model = load_model('compiler/SaveState.keras')

# Serialize model to JSON
model_json = model.to_json()
with open("compiler/model_architecture.json", "w") as json_file:
    json_file.write(model_json)

# Save model weights with the correct filename
model.save_weights('compiler/model_weights.weights.h5')
